In [10]:
# Import necessary libraries
import numpy as np
import yfinance as yf
from scipy.stats import norm

In [3]:
# set all the variables
symbol = "PLTR"
option_type = "call"
risk_free_rate = 0
time_to_expiry = 81/ 365
strike_price = 140

In [6]:
# Pull and prepare the market 
data = yf.download(symbol, period = "60d", interval = "1d", progress = False)
close_prices = data["Close"]
spot_price = close_prices.iat[-1,0]
returns = np.log(close_prices.PLTR / close_prices.PLTR.shift(1)).dropna()
volatility = np.std(returns) * np.sqrt(252)




C:\Users\rohit\AppData\Local\Temp\ipykernel_27784\800355049.py:2: FutureWarning: YF.download() has changed argument auto_adjust default to True
  data = yf.download(symbol, period = "60d", interval = "1d", progress = False)


In [7]:
# calculate black - scholes parameters 
d1 = (np.log(spot_price / strike_price) + (risk_free_rate + 0.5 * volatility ** 2) * time_to_expiry) / (volatility * np.sqrt(time_to_expiry))
d2 = d1 - volatility * np.sqrt(time_to_expiry)

In [11]:
# Compute option price and greeks 
if option_type == "call":
    option_price = spot_price * norm.cdf(d1) - strike_price * np.exp(-risk_free_rate * time_to_expiry) * norm.cdf(d2)
    delta = norm.cdf(d1)
else:
    option_price = strike_price * np.exp(-risk_free_rate * time_to_expiry) * norm.cdf(-d2) - spot_price * norm.cdf(-d1)
    delta = -norm.cdf(-d1)

gamma = norm.pdf(d1) / (spot_price * volatility * np.sqrt(time_to_expiry))

In [12]:
option_price, delta, gamma

(np.float64(15.902605697977513),
 np.float64(0.5200140999370128),
 np.float64(0.008688188736974346))

Option Price: ~$15.90
This represents the theoretical fair value of the option. For a call option, this is the premium you would pay to purchase the right to buy the underlying asset at the strike price.

Delta: ~0.52

Meaning: For every $1 increase in the underlying asset price, the option price will increase by approximately $0.52
Interpretation: This suggests the option is slightly in-the-money or at-the-money since delta is close to 0.5

Gamma: ~0.0087

Meaning: For every $1 increase in the underlying asset price, the delta will increase by approximately 0.0087